# 一、from 10 number FDDB-Annonations file to a txt File 

In [ ]:
import numpy as np
import cv2
import shutil
import os
from xml.dom.minidom import Document    

def widerFaceToRoidb():
    faceCount = 0
    boxCount = 0
    outFile = open('test_fddb.txt','a')
    for j in range(10):
        if j < 9:
            txtFileName = 'FDDB-folds/FDDB-fold-0%s.txt'%(str(j+1))
        else:
            txtFileName = 'FDDB-folds/FDDB-fold-10.txt'
        fp = open(txtFileName,'r')
        datas = fp.readlines()
        for i in range(len(datas)):
            data = datas[i].strip()
            data = '/jmu/jmuaia.tanghaom.top/data/User/admin/home/train/faceDetector/data/FDDB/' + data + '.jpg'+'\n'
            outFile.write(data)
    outFile.close()
widerFaceToRoidb()
    

# 二、 make FDDB dataset format from test result file. finishly.,make 10 txt file which like 10 train file' format.

In [ ]:
import os
outPath = 'pyramid-59500'
if not os.path.exists(outPath):
        os.makedirs(outPath)
fp = open('/jmu/jmuaia.tanghaom.top/data/User/admin/home/train/faceDetector/darknet-pyramid/results/Pyramid-59500-fddbface.txt','r')
#fp = open('/jmu/jmuaia.tanghaom.top/data/User/admin/home/train/faceDetector/darknet-v1.0/results/35000-fddbface.txt','r')
datas = fp.readlines()
result = {}
preImgName = datas[0].strip().split(' ')[0]
preImgName = preImgName.split('.jpg')[0]
preImgName = preImgName.replace('FDDB/','')
print preImgName
box =  datas[0].strip().split(' ')[1:]
box[3] = str(float(box[3]) - float(box[1]))
box[4] = str(float(box[4]) - float(box[2]))
score = box[0]
box[0:4] = box[1:5]
box[4] = score
print box
result[preImgName] = [box]
print(result)
print box
i = 1
count = 1
while i < len(datas):
    imageName = datas[i].strip().split(' ')[0]
    
    imageName = imageName.split('.jpg')[0]
    # 需要修改
    imageName = imageName.replace('/jmu/jmuaia.tanghaom.top/data/User/admin/home/train/faceDetector/data/FDDB/','')
    box =  datas[i].strip().split(' ')[1:]
    box[3] = str(float(box[3]) - float(box[1]))
    box[4] = str(float(box[4]) - float(box[2]))
    score = box[0]
    box[0:4] = box[1:5]
    box[4] = score
    #print box
    if  imageName == preImgName:
        result[imageName].append(box)
    else: 
        count += 1
        preImgName = imageName
        result[imageName] = [box]
    i+=1
   

fp.close()
for k in range(10):
    fileName = 'FDDB-folds/FDDB-fold-%s.txt'%(str("%02d"%(k+1)))
    outFile = open(fileName,'r')
    fileData = outFile.readlines()
    resultFile = open(outPath + '/fold-%s-out.txt'%(str("%02d"%(k+1))),'a')
    for data in fileData:
        data = data.strip()
        resultFile.write(data+'\n')
        imageName = data
        boxes = []
        if result.has_key(imageName):
            boxes = result[imageName]
        # print data,boxes
        boxNumber = len(boxes)
        resultFile.write(str(boxNumber)+'\n')
        for box in boxes:
            lines = box[0] + ' ' + box[1] + ' ' + box[2] + ' '+ box[3] + ' '+box[4]+'\n'
            resultFile.write(lines)
        if boxNumber == 0:
            print data
    resultFile.close()    
    outFile.close()
print(len(fileData))
#print len(result['img_207'])
print('image Number is :', count)

# 三、 draw roc curve

In [ ]:
mport numpy as np
import matplotlib.pyplot as plt
from pylab import *
import math
x = []
y = []
file = ['CE-35000/DiscROC.txt','FL-28300/DiscROC.txt','pyramid-59500/DiscROC.txt','mtcnn-result/DiscROC.txt','HR-DiscROC.txt','FastCNNDisc.txt','DeepIRDiscROC(2).txt']
labels = ['our CE-yolov3-spp-face','our FL-yolov3-spp-face','our-yolov3-spp-pyramid-face','mtcnn','CVPR2017 find tiny face','Fast RCNN','DeepIR']
maxTPR = []
for fileName in file:    
    fp = open(fileName,'r')
    datas= fp.readlines()
    x_tmp = []
    y_tmp = []
    for data in datas:
        data = data.strip().split(' ')
        y_tmp.append(float(data[0]))
        x_tmp.append(float(data[1]))
    y.append(y_tmp)
    x.append(x_tmp)
    fp.close
for i in range(len(x)):
   maxTPR.append(max(y[i]))
   plt.plot(np.array(x[i], dtype=int),np.array(y[i], dtype=float),label=labels[i] + '('+str(maxTPR[i])+')')
   
title='FDDB ROC Curve '
plt.title(title)
plt.xlabel('False Positive')
plt.ylabel('True Positive Rate')
plt.yticks(np.arange(0,1,0.05))
plt.ylim([0.0, 1.0])
plt.xlim([0.0, 2000.0])
plt.grid(ls='-.')
plt.legend()
plt.savefig('ROC-curve.png', dpi=300)